In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
api_token = {"username":"david10188","key":"3c6ba0ceb741ea66c0d22d10e0079cfc"}
import json
import zipfile
import os

if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 /root/.kaggle/kaggle.json

if not os.path.exists('/kaggle'):
    os.makedirs('/kaggle')
os.chdir('/kaggle')
!kaggle competitions download -c new-york-city-taxi-fare-prediction
!ls /kaggle 

 99% 1.54G/1.56G [00:14<00:00, 111MB/s]
100% 1.56G/1.56G [00:14<00:00, 113MB/s]
new-york-city-taxi-fare-prediction.zip


In [8]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
df_train = pd.read_csv(ZipFile('/kaggle/new-york-city-taxi-fare-prediction.zip').open('train.csv'),nrows=500000)
df_test = pd.read_csv(ZipFile('/kaggle/new-york-city-taxi-fare-prediction.zip').open('test.csv'))
train_Y = np.array(df_train['fare_amount'])
ids = df_test['key']

In [9]:
df_train = df_train.drop(['key', 'fare_amount'] , axis=1)
df_test = df_test.drop(['key'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()
df['distance_2D'] = ((df['dropoff_longitude'] - df['pickup_longitude'])**2 + (df['dropoff_latitude'] - df['pickup_latitude'])**2)**0.5
df = df.drop(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'pickup_datetime'], axis=1)
df.head()

,passenger_count,distance_2D
0,1,0.009436
1,1,0.079696
2,2,0.013674
3,1,0.025340
4,1,0.019470


In [10]:
df['passenger_count'] = df['passenger_count'].fillna(-1)
df['distance_2D'] = df['distance_2D'].fillna(-1)

In [11]:
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

from sklearn import datasets, metrics
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor()
reg.fit(train_X, train_Y)
pred = reg.predict(test_X)

In [12]:
data_path = '/content/drive/MyDrive/中大講師/中大講師/中大 - 天氣與人工智慧/111-1/期末考/data/'
pred = np.array(pred)
sub = pd.DataFrame({'key': ids, 'fare_amount': pred})
sub.to_csv(data_path + 'baseline/taxi_baseline_1111_finalexam.csv', index=False) 